### Importing all relevant libraries

In [ ]:
%matplotlib inline
from Functions import A1, A2, A3
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Defining the functions

In [ ]:
p = 14
run = 50

functions = [A1, A2, A3]

AE_df = pd.read_excel('AE_df.xlsx',index_col=0) #the analytical values

n = 2

k = 6

df = pd.DataFrame(sobol_seq.i4_sobol_generate(6*k,-1+2**p))

df_index = pd.read_csv('2019.10.08_index.csv', index_col=0)

df_r2 = pd.concat([df.T.reindex(df_index.iloc[r]).reset_index(drop=True).T for r in range(run)])

def scrambler(q,s):
    return q+s

### Running the algorithm

In [ ]:
ls = ['-',':','-.']
mk = ['+','X','v']

MAE_f = {f.__name__:[] for f in functions}
MAE_f_var = {f.__name__:[] for f in functions}

for f in functions:
    MAE_p = {2**pi:[] for pi in range(9,p+1)}
    MAE_p_var = {2**pi:[] for pi in range(9,p+1)}
    for ipi,pi in enumerate(range(9,p+1)):
        MAE_r = []
        for r in range(run):
            df_r =df_r2.iloc[r*(-1+2**p):(r+1)*(-1+2**p)]
            elementary_effect = []
            ea = pd.DataFrame(f(df_r.iloc[:,:k]))
            rs = [df_r.iloc[:,:2*k].copy() for j in range(k)]
            ee_df = [pd.Series() for j in range(k)]
            for j in range(k):
                rs[j][j] = rs[j][scrambler(j,k)]
                ee = pd.Series(0.5*(f(df_r.iloc[:-1+2**(pi-5),:k])-f(rs[j].iloc[:-1+2**(pi-5),:k]))**2,name=j)
                ee_df[j] = ee_df[j].append(ee)
                
            e_df = pd.concat(ee_df,axis=1)

            e_df_mean = e_df.mean()
            e_df_std = e_df.std().sort_values(ascending = False)

            e_filter = e_df_std.cumsum()/e_df_std.sum()
            
            if e_df_std.iloc[-1] > e_df_std.iloc[-2]/np.sqrt(2):  
                longitude = e_filter.sort_index()
            else:
                longitude = e_filter[:k-1].sort_index()

            for iq,q in enumerate(range(pi-4,pi+1)):
                for i in longitude.index:
                    ee = pd.Series(0.5*(f(rs[i].iloc[-1+2**(q-1):-1+2**q])-f(df_r.iloc[-1+2**(q-1):-1+2**q]))**2,name=i)
                    ee_df[i] = ee_df[i].append(ee)
                e_df = pd.concat(ee_df,axis=1)

                e_df_mean = e_df.mean()
                e_df_std = e_df.std().sort_values(ascending = False)

                e_filter = e_df_std.cumsum()/e_df_std.sum()
                if e_df_std.iloc[k-iq-2]/(np.sqrt(2)) > e_df_std.iloc[k-iq-1] and q < pi-1:
                    longitude = e_filter[:k-iq-1].sort_index()
                elif e_df_std.iloc[k-iq-2]/(np.sqrt(2)) < e_df_std.iloc[k-iq-1] and q < pi-1:
                    longitude = e_df_std[e_df_std>e_df_std.iloc[k-iq-2]/(np.sqrt(2))].sort_index()
                elif e_df_std.iloc[k-iq-2]/(np.sqrt(2)) > e_df_std.iloc[k-iq-1] and q == pi-1 and \
                (e_df.count(axis=1)+1).cumsum().iloc[-1] < (k+1-3)*(-1+2**(pi-1)):
                    longitude = e_filter[:2].sort_index()
                elif e_df_std.iloc[k-iq-2]/(np.sqrt(2)) > e_df_std.iloc[k-iq-1] and q == pi-1 and \
                (e_df.count(axis=1)+1).cumsum().iloc[-1] < (k+1-2)*(-1+2**(pi-1)):
                    longitude = e_filter[:1].sort_index()
                elif q == pi-1 and (e_df.count(axis=1)+1).cumsum().iloc[-1] > (k+1-2)*(-1+2**(pi-1)):
                    break    
                elif q == pi-1:
                    longitude = e_filter[:np.int(np.floor(((k+1)*(-1+2**(pi-1))-(e_df.count(axis=1)+1).cumsum().iloc[-1])/\
                                          (-1+2**(pi-1))))-1].sort_index()
                    
            Var_df = f(df_r.iloc[:len(e_df)]).expanding(1).var(ddof=0)
            
            T = e_df.expanding(1).mean()

            AE = np.abs((T.T.div(Var_df)).T - AE_df.loc[f.__name__])

            MAE = AE.mean(axis=1)
            MAE.index = (e_df.count(axis=1)+1).cumsum()
            MAE_r.append(MAE)
        MAE_p[2**pi] = pd.concat(MAE_r,axis=1).interpolate().mean(axis=1)
        MAE_p_var[2**pi] = pd.concat(MAE_r,axis=1).interpolate().iloc[-1]
    MAE_f[f.__name__] = MAE_p
    
    Saltenis_scrambled = pd.read_excel('Saltenis_asym.xlsx',sheet_name=f.__name__,index_col=0).iloc[[-1+2**pi \
                                                                                                     for pi in range(8,p)]]

    dataSets = [Saltenis_scrambled,pd.concat([MAE_f[f.__name__][c].dropna().iloc[-1:] for c in MAE_f[f.__name__]])]

    dSnames = ['Saltenis','New_Estimator_Sobol_reallocation_adaptive']

    dS_dic = dict(zip(dSnames,dataSets))

    co = -1
    for dk, dv in dS_dic.items():
        co +=1
        x_vals = dv.index
        y1 = dv.values
        ax = plt.gca()
        ax.scatter(x_vals, y1, c='k', marker = mk[co], linestyle=ls[co],label = dk)
        ax.set_xscale('log')
        ax.set_yscale('log')
    plt.xlabel('Cost')
    plt.ylabel('MAE')
    plt.title(f.__name__)
    plt.legend()
    plt.show()